In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = "shahnameh.txt"

In [4]:
text = open(path_to_file, "rb").read().decode(encoding= "utf-8")
print ("Length of text: {} characters".format(len(text)))

Length of text: 2653849 characters


In [5]:
print(text[:250])

|به نام خداوند جان و خرد
|کزین برتر اندیشه برنگذرد
|خداوند نام و خداوند جای
|خداوند روزی ده رهنمای
|خداوند کیوان و گردان سپهر
|فروزنده ماه و ناهید و مهر
|ز نام و نشان و گمان برترست
|نگارندهٔ بر شده پیکرست
|به بینندگان آفریننده را
|نبینی مرنجان دو بین


In [6]:
vocab = sorted(set(text))
print ("{} unique characters".format(len(vocab)))

48 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print("{")
for char,_ in zip(char2idx, range(20)):
    print("  {:4s}: {:3d},".format(repr(char), char2idx[char]))
print("  ...\n}")

{
  '\n':   0,
  ' ' :   1,
  '(' :   2,
  ')' :   3,
  '|' :   4,
  '«' :   5,
  '»' :   6,
  '،' :   7,
  '؟' :   8,
  'ء' :   9,
  'آ' :  10,
  'أ' :  11,
  'ؤ' :  12,
  'ئ' :  13,
  'ا' :  14,
  'ب' :  15,
  'ت' :  16,
  'ث' :  17,
  'ج' :  18,
  'ح' :  19,
  ...
}


In [9]:
print ("{} ---- characters mapped to int ---- > {}".format(repr(text[:13]), text_as_int[:13]))

'|به نام خداون' ---- characters mapped to int ---- > [ 4 15 38  1 37 14 36  1 20 21 14 39 37]


In [10]:
seq_length = 100

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

|
ب
ه
 
ن


In [14]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
    print(repr("".join(idx2char[item.numpy()])))
    print("***" * 5)

'|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'
***************
'داوند کیوان و گردان سپهر\n|فروزنده ماه و ناهید و مهر\n|ز نام و نشان و گمان برترست\n|نگارندهٔ بر شده پیکر'
***************
'ست\n|به بینندگان آفریننده را\n|نبینی مرنجان دو بیننده را\n|نیابد بدو نیز اندیشه راه\n|که او برتر از نام و'
***************
' از جایگاه\n|سخن هر چه زین گوهران بگذرد\n|نیابد بدو راه جان و خرد\n|خرد گر سخن برگزیند همی\n|همان را گزین'
***************
'د که بیند همی\n|ستودن نداند کس او را چو هست\n|میان بندگی را ببایدت بست\n|خرد را و جان را همی سنجد اوی\n|د'
***************


In [15]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
  print ("Input data: ", repr("".join(idx2char[input_example.numpy()])))
  print ("Target data:", repr("".join(idx2char[target_example.numpy()])))

Input data:  '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'
Target data: 'به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|خ'


In [17]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 4 ('|')
  expected output: 15 ('ب')
Step    1
  input: 15 ('ب')
  expected output: 38 ('ه')
Step    2
  input: 38 ('ه')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 37 ('ن')
Step    4
  input: 37 ('ن')
  expected output: 14 ('ا')


In [18]:
BATCH_SIZE = 64
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [19]:
vocab_size = len(vocab)
embedding_dim = 25
rnn_units = 1024

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
          tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
          tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer = "glorot_uniform"),
          tf.keras.layers.Dense(vocab_size)])
  return model

In [21]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim = embedding_dim,
                    rnn_units = rnn_units,
                    batch_size = BATCH_SIZE)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model.predict(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 48) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 25)            1200      
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3228672   
_________________________________________________________________
dense (Dense)                (64, None, 48)            49200     
Total params: 3,279,072
Trainable params: 3,279,072
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices,axis = -1).numpy()

In [25]:
sampled_indices

array([38, 47, 10, 11, 21, 29, 37, 41, 13, 35, 41, 23, 12, 21, 41, 15, 47,
       29,  3, 10, 42, 33, 12, 13, 18,  4, 35, 27, 41, 16,  8, 45, 15, 43,
        8, 40, 28, 35, 29,  3,  5,  8, 17, 32, 10, 44, 25, 24, 45, 24, 26,
       40, 19,  8,  6, 45, 25,  3, 31, 13, 28, 12,  6, 46, 22, 45, 12, 33,
       42, 33,  0, 15, 32, 36, 45, 22, 33, 44, 43, 33, 36, 36,  9, 47,  4,
       18,  0, 31, 33, 33,  0, 45, 37, 15, 44, 22, 30, 10, 29, 28],
      dtype=int64)

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '|به نام خداوند جان و خرد\n|کزین برتر اندیشه برنگذرد\n|خداوند نام و خداوند جای\n|خداوند روزی ده رهنمای\n|'

Next Char Predictions: 
 'ه\u200cآأدطنپئلپرؤدپب\u200cط)آچفؤئج|لصپت؟گبژ؟ٔضلط)«؟ثغآکسزگزشٔح؟»گس)عئضؤ»یذگؤفچف\nبغمگذفکژفممء\u200c|ج\nعفف\nگنبکذظآطض'


In [27]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [28]:
model.compile(optimizer = "adam", loss=loss)

In [29]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only = True)

In [30]:
Epoch = 10

In [31]:
history = model.fit(dataset, epochs = Epoch, callbacks = [checkpoint_callback])

Epoch 1/10
410/410 [==============================] - 129s 314ms/step - loss: 2.4480
Epoch 2/10
410/410 [==============================] - 149s 363ms/step - loss: 1.8074
Epoch 3/10
410/410 [==============================] - 152s 371ms/step - loss: 1.5528
Epoch 4/10
410/410 [==============================] - 154s 375ms/step - loss: 1.4235
Epoch 5/10
410/410 [==============================] - 154s 377ms/step - loss: 1.3444
Epoch 6/10
410/410 [==============================] - 157s 382ms/step - loss: 1.2861
Epoch 7/10
410/410 [==============================] - 157s 383ms/step - loss: 1.2370
Epoch 8/10
410/410 [==============================] - 136s 331ms/step - loss: 1.1920
Epoch 9/10
410/410 [==============================] - 147s 359ms/step - loss: 1.1487
Epoch 10/10
410/410 [==============================] - 152s 370ms/step - loss: 1.1066


In [32]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 25)             1200      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3228672   
_________________________________________________________________
dense_1 (Dense)              (1, None, 48)             49200     
Total params: 3,279,072
Trainable params: 3,279,072
Non-trainable params: 0
_________________________________________________________________


In [35]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions 
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

  return (start_string + "".join(text_generated))

In [38]:
print(generate_text(model, start_string = u"به نام خدا"))

به نام خدایست نستوه تو
|به گیتی نماید نگارد ز بتاختی
|وزان پس چنین تا برآرم بماه
|صد آن تختها برکشمد از تو بر تن خویش یابی به خون
|ز شادی شگفتی که بیکار گشت
|دوماه
|کجا آن همه ریز کردم همی
|ز تخم بد و باژ و پر بوی مهر
|شنیده تخت باژی چو کوه بزرگ
|بدست سخن گوی برخاستند
|به زندان بیاوردش از جنگ جفت
|یکی دیگر آنگه که تن بگذرد
|من آن تخت راخسر بر تنگ هنگام موسن شود
|سربخت این را که پوشیده‌ام
|سراسان کنم داد و دانندگان
|گلاب و عنان برگرفتند راه
|نماند به رستم که لشکر براند
|چه افگند دینار و گرمان به دست
|چو ارجات داری خرامید یاد
|که نزد کزت بر تو بر خاک روی
|شهنشاه بینندهٔ رخش بروخون
|تو گفتی همی درکشید این سخن
|سواری بر اب گوهرنگار
|صزو تن به پا اندر آویختست
|نه زین باره و گردیه را بدست
|به خون خسره آیید گفتار من
|نگردد به بازد اسیدش تخل به درد
|سوی حلبهاد آن سه زر
|سپاس از دبیرو ستم
|همی دشمنندان او تخت را نو نمرد
|هرآنکس که او دشمن ایمن ببین
|بدو گفت بهرام چون بر روان
|یبا پیرسر گفت زن پر ز خون
|نگه کرده و از بلت خسرو شوردار
|بدآنید تاوان به ایران تویی
|
|ار و دوبست و زه برکشد
|فروشد نه 